In [ ]:
!pip install pyDOE2
!pip install pyopenms numpy matplotlib
# Import necessary libraries
import numpy as np
import pandas as pd
import scipy.io as sio
import os
from pyDOE2 import ccdesign
import statsmodels.api as sm
from statsmodels.formula.api import ols


# Define parameter levels
parameters = {
    'SignalIntensity': [2500, 12500, 25000],
    'MassError': [0.05, 0.1],
    'Occurrences': [10, 15, 20],

}

# Define the number of factors
num_factors = len(parameters)

# Generate the Central Composite Design (CCD)
ccd_design = ccdesign(num_factors, center=(2, 0), alpha='orthogonal')

# Map CCD design matrix to actual factor levels
def map_ccd_to_levels(ccd_design, parameters):
    num_levels = {param: len(levels) for param, levels in parameters.items()}
    mapped_matrix = np.zeros_like(ccd_design)

    for i, (param, levels) in enumerate(parameters.items()):
        levels = np.array(levels)
        num = num_levels[param]

        if num == 2:
            # Map -1, 0, 1 to the two levels
            mapped_matrix[:, i] = np.where(ccd_design[:, i] == -1, levels[0],
                                           np.where(ccd_design[:, i] == 1, levels[-1], levels[0]))
        elif num == 3:
            # Map -1, 0, 1 to the three levels by ensuring exact levels
            mapped_matrix[:, i] = np.interp(ccd_design[:, i], [-1, 0, 1], levels)
        else:
            raise ValueError("Number of levels must be 2 or 3 for this mapping.")

    return mapped_matrix

# Map the CCD design to actual parameter levels
mapped_design = map_ccd_to_levels(ccd_design, parameters)
design_df = pd.DataFrame(mapped_design, columns=parameters.keys())

# Ensure all values are within the specified levels
for param, levels in parameters.items():
    unique_values = np.unique(design_df[param])
    print(f"Unique values for {param}: {unique_values}")
    if not np.all(np.isin(unique_values, levels)):
        raise ValueError(f"Some values for {param} are not in the specified levels.")

# Print the design matrix
print("Central Composite Design Matrix:")
print(design_df.to_string(index=False))

# Print the number of runs
num_runs = len(design_df)
print(f"\nNumber of runs in the Central Composite Design: {num_runs}")

# Save the design matrix to a CSV file
csv_file_path = "write your directory"
design_df.to_csv(csv_file_path, index=False)

print(f"\nDesign matrix saved to: {csv_file_path}")
